# Stochastic Unit Commitment Problem

In [1]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg

from sddip import config, storage, utils, parameters
from sddip import dualsolver, ucmodel, scenarios

## Data Processing

In [2]:
test_case = "WB2"

params = parameters.Parameters(test_case)

bus_df = params.bus_df
branch_df = params.bus_df
gen_df = params.gen_df
gen_cost_df = params.gen_cost_df
gen_sup_df = params.gen_sup_df
storage_df = params.storage_df
scenario_df = params.scenario_df

### Bus Data

In [3]:
bus_df

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1,3,0,0,0,0,1,0.964,0,0,1,1.05,0.95
1,2,1,350,-350,0,0,1,1.000,-65,0,1,1.05,0.95


### Branch Data

In [4]:
branch_df

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1,3,0,0,0,0,1,0.964,0,0,1,1.05,0.95
1,2,1,350,-350,0,0,1,1.000,-65,0,1,1.05,0.95


### Generator Data

In [5]:
gen_df

,bus,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,...,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf
0,1,400,100,400,-400,0.964,100,1,600,0,...,0,0,0,0,0,0,0,0,0,0


### Generator Cost Data

In [6]:
gen_cost_df

,type,startup,shutdown,n,c2,c1,c0
0,2,0,0,3,0,2,0


### Storage Data

In [8]:
storage_df

,bus,Rc,Rdc,Effc,Effdc,SOC
0,1,25,25,1,1,100


In [7]:
scenario_df

,t,n,p,Pd1,Pd2,Re1,Re2
0,1,1,1.0,0,174.047689,0,58.841980
1,2,1,0.5,0,324.065867,0,45.141510
2,2,2,0.5,0,330.583183,0,54.413346
3,3,1,0.5,0,320.983740,0,37.409633
4,3,2,0.5,0,373.451597,0,44.115535


### Power Transfer Distribution Factor

In [109]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

ptdf = np.insert(ptdf, ref_bus-1, 0, axis=1)

ptdf

array([[ 0.        , -0.39566454, -0.72503406],
       [ 0.        , -0.39566454,  0.27496594],
       [ 0.        , -0.60433546, -0.27496594]])

### Parameter Organization

In [110]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

# Lists of generators at each bus
#
# Example: [[0,1], [], [2]]
# Generator 1 & 2 are located at bus 1
# No Generator is located at bus 2
# Generator 3 is located at bus 3
gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1

# TODO Add ramp rate limits
rg_up_max = np.full(n_gens, 1000)
rg_down_max = np.full(n_gens, 1000)
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

prob = []
p_d = []

for t in range(n_stages):
    stage_df = scenario_df[scenario_df["t"] == t+1]
    p_d.append(stage_df[scenario_df.columns[scenario_df.columns.to_series().str.contains('Pd')]].values.tolist())
    prob.append(stage_df["p"].values.tolist())


########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = [np.array(prob[t]).dot(np.array(p_d[t])) for t in range(n_stages)]

In [111]:
# prob[t][n]
# Probability of realization n at stage t
#prob

[[1.0], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5]]

In [112]:
# p_d[t][n][b]
# Demand in stage t and realization n at bus b
#p_d

[[[44.54372063015073, 39.87236623381583, 36.93197251275404]],
 [[62.54501068568694, 61.92614621908863, 50.108444390940704],
  [65.15880071905508, 64.42778836961988, 57.203042112305226]],
 [[112.95342858934262, 100.26378409014924, 83.68709047284852],
  [105.88728210075874, 98.82902687421333, 87.81655289255107]],
 [[102.47359712356636, 102.86368574222556, 95.7770332692824],
  [97.60080286189832, 102.11013441557157, 85.05346559544873]],
 [[100.25006136356586, 95.53875225838196, 84.69123942554963],
  [103.10516072236364, 97.5092131933629, 92.6944656222324]],
 [[110.31905816594666, 119.95213524929262, 102.05904507775809],
  [109.4285821721908, 102.5049435468718, 87.92290719259505]]]

In [113]:
# ex_pd[t][b]
# Expected demand in stage t at bus b
#ex_pd

[array([44.54372063, 39.87236623, 36.93197251]),
 array([63.8519057 , 63.17696729, 53.65574325]),
 array([109.42035535,  99.54640548,  85.75182168]),
 array([100.03719999, 102.48691008,  90.41524943]),
 array([101.67761104,  96.52398273,  88.69285252]),
 array([109.87382017, 111.2285394 ,  94.99097614])]

## SDDiP

In [114]:
# Result keys
x_key = "x"
y_key = "y"
x_bs_key = "x_bs"
primal_solution_keys = [x_key, y_key, x_bs_key]

dv_key = "dual_value"
dm_key = "dual_multiplier"
dual_solution_keys = [dv_key, dm_key]

ci_key = "intercepts"
cg_key = "gradients"
bm_key = "multipliers"
cut_coefficient_keys = [ci_key, cg_key, bm_key]


# Result storage
ps_storage = storage.ResultStorage(primal_solution_keys, "primal")
ds_storage = storage.ResultStorage(dual_solution_keys, "dual")
cc_storage = storage.ResultStorage(cut_coefficient_keys, "cuts")


In [115]:
global_sos = True

### Forward pass

In [116]:
penalty = 10000
cost_coeffs = gc.tolist() + suc.tolist() + sdc.tolist() + [penalty]*2
theta_lb = 0

min_up_times = [2]*n_gens
min_down_times = [2]*n_gens
backsight_periods = [max(ut, dt) for ut, dt in zip(min_up_times, min_down_times)]

x_trial_point = [0]*n_gens
y_trial_point = [0]*n_gens
x_bs_trial_point = [[0]*periods for periods in backsight_periods]
soc_trial_point = params.init_soc_trial_point

# Sampling
n_samples = 1
samples = scenarios.ScenarioSampler(n_stages, 2).generate_samples(n_samples)

v_opt_k = []
i = 0
for k in range(n_samples):
    v_opt_k.append(0)
    print(f"k={k}")
    for t, n in zip(range(n_stages), samples[k]):

        # Create forward model
        uc_fw = ucmodel.ForwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus, backsight_periods)

        uc_fw.add_objective(cost_coeffs)

        uc_fw.add_balance_constraints(sum(params.p_d[t][n]), sum(params.re[t][n]))

        uc_fw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

        uc_fw.add_generator_constraints(pg_min, pg_max)

        uc_fw.add_startup_shutdown_constraints()

        uc_fw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

        uc_fw.add_up_down_time_constraints(min_up_times, min_up_times)

        uc_fw.add_copy_constraints(x_trial_point, y_trial_point, x_bs_trial_point)

        #TODO Lower bound
        uc_fw.add_cut_lower_bound(theta_lb)
        
        if i>0:
            cut_coefficients = cc_storage.get_stage_result(t)
            uc_fw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key])
        
        # Solve problem
        uc_fw.disable_output()
        uc_fw.model.optimize()
        model = uc_fw.model
        uc_fw.model.printAttr("X")

        # Store xtik, ytik, ztik, vtik
        x_kt = [x_g.x for x_g in uc_fw.x]
        y_kt = [y_g.x for y_g in uc_fw.y]
        x_bs_kt = [[x_bs_g.x for x_bs_g in x_bs] for x_bs in uc_fw.x_bs ]
        z_x_kt = [z_g.x for z_g in uc_fw.z_x]
        z_y_kt = [z_g.x for z_g in uc_fw.z_y]
        s_up_kt = [s_up_g.x for s_up_g in uc_fw.s_up]
        s_down_kt = [s_down_g.x for s_down_g in uc_fw.s_down]
        # Value of stage t objective function
        print(f"Objective value t={t}, n={n}: {uc_fw.model.getObjective().getValue()}")
        v_opt_kt = uc_fw.model.getObjective().getValue() - uc_fw.theta.x

        v_opt_k[-1] += v_opt_kt

        # New trial point
        x_trial_point = x_kt
        y_trial_point = y_kt
        x_bs_trial_point = [[x_trial_point[g]]+x_bs_kt[g][:-1] for g in range(n_gens)]

        ps_dict = ps_storage.create_empty_result_dict()
        ps_dict[x_key] = x_kt
        ps_dict[y_key] = y_kt
        ps_dict[x_bs_key] = x_bs_trial_point
        
        ps_storage.add_result(i, k, t, ps_dict)

k=0
Objective value t=0, n=0: 150.61767125206472
[[0.0, -0.0], [1.0, -0.0]]
Objective value t=1, n=1: 224.14755744117622
[[0.0, -0.0], [1.0, 1.0]]
Objective value t=2, n=0: 356.28516378280847
[[0.0, -0.0], [1.0, 1.0]]
Objective value t=3, n=0: 370.5715806753716
[[1.0, -0.0], [1.0, 1.0]]
Objective value t=4, n=1: 351.9706074455507
[[1.0, 1.0], [1.0, 1.0]]
Objective value t=5, n=0: 521.6511924649867
[[1.0, 1.0], [1.0, 1.0]]


#### Statistical Upper Bound

In [118]:
v_opt_k = np.array(v_opt_k)

v_mean = np.mean(v_opt_k)
v_std = np.std(v_opt_k)
alpha = 0.05

# Lower limit of the confidence interval
v_upper = v_mean + stats.norm.ppf(alpha/2)*v_std/np.sqrt(n_samples)
v_upper

1975.2437730619586

In [119]:
# Upper limit of the confidence interval
v_mean - stats.norm.ppf(alpha/2)*v_std/np.sqrt(n_samples)

1975.2437730619586

In [120]:
primal_solution_df = ps_storage.to_dataframe()
primal_solution_df

x                            y                        x_bs
i k t                                                                     
0 0 0  [0.0, 1.0]    [0.0, 121.34805937672061]  [[0.0, -0.0], [1.0, -0.0]]
    1  [0.0, 1.0]    [0.0, 186.78963120098018]   [[0.0, -0.0], [1.0, 1.0]]
    2  [0.0, 1.0]     [0.0, 296.9043031523404]   [[0.0, -0.0], [1.0, 1.0]]
    3  [1.0, 1.0]  [1.1143161350743185, 300.0]   [[1.0, -0.0], [1.0, 1.0]]
    4  [1.0, 1.0]    [0.0, 293.30883953795893]    [[1.0, 1.0], [1.0, 1.0]]
    5  [1.0, 1.0]   [32.33023849299735, 300.0]    [[1.0, 1.0], [1.0, 1.0]]

### Backward pass

In [121]:
########################################################################################################################
# Backward pass
########################################################################################################################


binarizer = utils.Binarizer()

sg_method = dualsolver.SubgradientMethod(max_iterations=10)

cut_lower_bounds = [theta_lb]*n_stages
v_opt_k =[]


i = 0

for t in reversed(range(1,n_stages)):
    n_realizations = n_nodes_per_stage[t]
    for k in range(n_samples):
        ds_dict = ds_storage.create_empty_result_dict()
        cc_dict = cc_storage.create_empty_result_dict()
        
        for n in range(n_realizations):

            # TODO Binarization
            bin_vars = []
            bin_multipliers = []
            if t>0:
                y_float_vars = ps_storage.get_result(i,k,t-1)[y_key]
                x_binary_trial_point = ps_storage.get_result(i,k,t-1)[x_key]
                x_bs_binary_trial_point = ps_storage.get_result(i,k,t-1)[x_bs_key]
            else:
                #TODO Approximation needed?
                # Might lead to active penalty
                y_float_vars = np.zeros(n_gens)
                x_binary_trial_point = np.zeros(n_gens)
                x_bs_binary_trial_point = [[0]*periods for periods in backsight_periods]
            
            for j in range(len(y_float_vars)):
                #new_vars, new_multipliers = binarizer.binary_expansion(float_vars[j], upper_bound=pg_max, precision=0.5)
                new_vars, new_multipliers = binarizer.binary_expansion_from_n_binaries(y_float_vars[j], pg_max[j], 14)
                bin_vars += new_vars
                bin_multipliers.append(new_multipliers) 


            y_binary_trial_point = bin_vars
            y_binary_trial_multipliers = linalg.block_diag(*bin_multipliers)


            uc_bw = ucmodel.BackwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus, backsight_periods)

            uc_bw.add_objective(cost_coeffs)

            uc_bw.add_balance_constraints(sum(p_d[t][n]))

            uc_bw.add_generator_constraints(pg_min, pg_max)

            uc_bw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

            uc_bw.add_startup_shutdown_constraints()

            uc_bw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

            uc_bw.add_relaxation(x_binary_trial_point, y_binary_trial_point, x_bs_binary_trial_point)

            uc_bw.add_copy_constraints(y_binary_trial_multipliers)

            uc_fw.add_up_down_time_constraints(min_up_times, min_up_times)

            uc_bw.add_cut_lower_bound(cut_lower_bounds[t])
            
            if t < n_stages-1:
                cut_coefficients = cc_storage.get_stage_result(t)
                uc_bw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key], big_m=10**25, sos=global_sos)

            objective_terms = uc_bw.objective_terms
            relaxed_terms = uc_bw.relaxed_terms
            
            # Solve problem with subgradient method
            uc_bw.disable_output()
            #uc_bw.model.optimize()
            #uc_bw.model.display()
            
            display_iteration_info = False
            if display_iteration_info:
                print()
                print("#"*20)
                print(f"# t={t}, n={n}")
                print("#"*20)

                print(f"Trial point: {x_binary_trial_point+y_binary_trial_point}", "\n")
                sg_method.output_flag = True
                sg_method.output_verbose = True
                        
            model, sg_results = sg_method.solve(uc_bw.model, objective_terms, relaxed_terms, 100)
            
            # Dual value and multiplier for each realization
            binary_trial_point = x_binary_trial_point + y_binary_trial_point + [x_bs_g for x_bs in x_bs_binary_trial_point for x_bs_g in x_bs]
            dual_multipliers = sg_results.multipliers.tolist()
            dual_value = sg_results.obj_value - np.array(dual_multipliers).dot(binary_trial_point)

            ds_dict[dv_key].append(dual_value)
            ds_dict[dm_key].append(dual_multipliers)
            
        
        ds_storage.add_result(i, k, t, ds_dict)

        # Calculate and store cut coefficients
        probabilities = prob[t]        
        intercept = np.array(probabilities).dot(np.array(ds_dict[dv_key]))
        gradient = np.array(probabilities).dot(np.array(ds_dict[dm_key]))

        cc_dict[ci_key] = intercept.tolist()
        cc_dict[cg_key] = gradient.tolist()
        cc_dict[bm_key] = y_binary_trial_multipliers

        if t > 0 : cc_storage.add_result(i, k, t-1, cc_dict)
                

In [122]:
########################################################################################################################
# Backward Primal solutions
########################################################################################################################


binarizer = utils.Binarizer()

sg_method = dualsolver.SubgradientMethod(max_iterations=10)

cut_lower_bounds = [theta_lb]*n_stages
v_opt_k =[]


i = 0
for t in reversed(range(1,n_stages)):
    n_realizations = n_nodes_per_stage[t]
    for k in range(n_samples):
        ds_dict = ds_storage.create_empty_result_dict()
        cc_dict = cc_storage.create_empty_result_dict()
        
        for n in range(n_realizations):

            # TODO Binarization
            bin_vars = []
            bin_multipliers = []
            if t>0:
                float_vars = ps_storage.get_result(i,k,t-1)[y_key]
                x_binary_trial_point = ps_storage.get_result(i,k,t-1)[x_key]
                x_bs_binary_trial_point = ps_storage.get_result(i,k,t-1)[x_bs_key]
            else:
                #TODO Approximation needed?
                # Might lead to active penalty
                float_vars = np.zeros(n_gens)
                x_binary_trial_point = np.zeros(n_gens)
                x_bs_binary_trial_point = [[0]*periods for periods in backsight_periods]
            
            for j in range(len(float_vars)):
                #new_vars, new_multipliers = binarizer.binary_expansion(float_vars[j], upper_bound=pg_max[j], precision=0.5)
                new_multipliers = binarizer.calc_binary_multipliers_from_n_binaries(pg_max[j], 14)
                new_vars, new_multipliers = binarizer.get_best_binary_approximation(float_vars[j], new_multipliers)
                bin_vars += new_vars
                bin_multipliers.append(new_multipliers) 


            y_binary_trial_point = bin_vars
            y_binary_trial_multipliers = linalg.block_diag(*bin_multipliers)


            uc_bw = ucmodel.BackwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus, backsight_periods)

            uc_bw.add_objective(cost_coeffs)

            uc_bw.add_balance_constraints(sum(p_d[t][n]))

            uc_bw.add_generator_constraints(pg_min, pg_max)

            uc_bw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

            uc_bw.add_startup_shutdown_constraints()

            uc_bw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

            uc_bw.add_up_down_time_constraints(min_up_times, min_down_times)

            uc_bw.add_relaxation(x_binary_trial_point, y_binary_trial_point, x_bs_binary_trial_point)

            uc_bw.add_copy_constraints(y_binary_trial_multipliers)

            uc_bw.add_cut_lower_bound(cut_lower_bounds[t])
            
            if t < n_stages-1:
                cut_coefficients = cc_storage.get_stage_result(t)
                uc_bw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key], big_m=10**25, sos=global_sos)

            objective_terms = uc_bw.objective_terms
            relaxed_terms = uc_bw.relaxed_terms
            
            for rt in relaxed_terms:
                uc_bw.model.addConstr(rt==0)
            
            uc_bw.model.optimize()
            obj_val = uc_bw.model.getObjective().getValue()
            print(f"Primal optimal value t={t}, n={n}: {obj_val}")

Primal optimal value t=5, n=0: 521.6511924649867
Primal optimal value t=5, n=1: 359.82771949398915
Primal optimal value t=4, n=0: 777.2090708015628
Primal optimal value t=4, n=1: 792.612000517764
Primal optimal value t=3, n=0: 1154.9918259969058
Primal optimal value t=3, n=1: 1126.117610342001
Primal optimal value t=2, n=0: 1496.4476483671472
Primal optimal value t=2, n=1: 1491.1963106059025
Primal optimal value t=1, n=0: 1702.7205117957105
Primal optimal value t=1, n=1: 1717.3885106001683


In [123]:
# One dual value and one set of dual multipliers for each realization
ds_storage.to_dataframe()


dual_value  \
i k t                                             
0 0 5    [519.297798843329, 359.82771949398915]   
    4    [773.5809223015074, 788.9838520182768]   
    3  [1152.2901926510078, 1123.4159769955347]   
    2   [1493.9054707011667, 1488.654132939922]   
    1    [1700.68727835377, 1715.3552771582276]   

                                         dual_multiplier  
i k t                                                     
0 0 5  [[0.0, 0.0, -0.19611613513818404, -0.196116135...  
    4  [[0.0, 0.0, 0.0, -0.19611613513818404, 0.0, 0....  
    3  [[-5.639663685229833, 0.0, -0.1924500897298752...  
    2  [[-3.196116135138184, 0.0, -0.1961161351381840...  
    1  [[-3.2041241452319316, 0.0, -0.204124145231931...

In [124]:
cc_storage.to_dataframe()


intercepts                                          gradients  \
i k t                                                                   
0 0 4   439.562759  [0.0, 0.0, -0.09805806756909202, -0.0980580675...   
    3   781.282387  [0.0, 0.0, 0.0, -0.19611613513818404, 0.0, 0.0...   
    2  1137.853085  [-2.9160568874798543, 0.0, -0.1924500897298752...   
    1  1491.279802  [-3.196116135138184, 0.0, -0.19611613513818404...   
    0  1708.021278  [-3.2041241452319316, 0.0, -0.2041241452319315...   

                                             multipliers  
i k t                                                     
0 0 4  [[0.018311664530305805, 0.03662332906061161, 0...  
    3  [[0.018311664530305805, 0.03662332906061161, 0...  
    2  [[0.018311664530305805, 0.03662332906061161, 0...  
    1  [[0.018311664530305805, 0.03662332906061161, 0...  
    0  [[0.018311664530305805, 0.03662332906061161, 0...

### Lower Bound Update

In [125]:
i = 1
t = 0
n = 0

x_trial_point = [0]*n_gens
y_trial_point = [0]*n_gens
x_bs_trial_point = [[0]*periods for periods in backsight_periods]


# Create forward model
uc_fw = ucmodel.ForwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus, backsight_periods)

uc_fw.add_objective(cost_coeffs)

uc_fw.add_balance_constraints(sum(p_d[t][n]))

uc_fw.add_power_flow_constraints(ptdf, pl_max, p_d[t][n])

uc_fw.add_generator_constraints(pg_min, pg_max)

uc_fw.add_startup_shutdown_constraints()

uc_fw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

uc_fw.add_up_down_time_constraints(min_down_times, min_up_times)

uc_fw.add_copy_constraints(x_trial_point, y_trial_point, x_bs_trial_point)

#TODO Lower bound
uc_fw.add_cut_lower_bound(theta_lb)

if i>0:
    cut_coefficients = cc_storage.get_stage_result(t)
    uc_fw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key], big_m=10**25, sos=global_sos)

# Solve problem
#uc_fw.disable_output()
uc_fw.model.optimize()
uc_fw.model.printAttr("X")
    


# Store xtik, ytik, ztik, vtik
x_kt = [x_g.x for x_g in uc_fw.x]
y_kt = [y_g.x for y_g in uc_fw.y]
z_x_kt = [z_g.x for z_g in uc_fw.z_x]
z_y_kt = [z_g.x for z_g in uc_fw.z_y]
s_up_kt = [s_up_g.x for s_up_g in uc_fw.s_up]
s_down_kt = [s_down_g.x for s_down_g in uc_fw.s_down]

# Value of stage t objective function
v_lower = uc_fw.model.getObjective().getValue()

v_lower
#TODO : Problem cut ist keine Funktion in y, sondern scalar

1860.2719330761556

In [126]:
# model.setParam("DualReductions", 0)
# model.reset()
# model.optimize()

In [127]:
# model.computeIIS()
# if model.IISMinimal:
#     print("IIS is minimal.")
# else:
#     print("IIS is not minimal.")
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s'%model.constrName)

In [128]:
# model.write("iis"+".ilp")

In [129]:
#df = ps_storage.to_dataframe()
#df

In [1]:
[1,3,4] + [5,6,7]

[1, 3, 4, 5, 6, 7]